# Working with preprocessing layers
# 使用预处理层

## Keras preprocessing layers
The Keras preprocessing layers API allows developers to build Keras-native input processing pipelines. These input processing pipelines can be used as independent preprocessing code in non-Keras workflows, combined directly with Keras models, and exported as part of a Keras SavedModel.

With Keras preprocessing layers, you can build and export models that are truly end-to-end: models that accept raw images or raw structured data as input; models that handle feature normalization or feature value indexing on their own.

## Keras预处理层
Keras预处理层API允许开发者建立Keras-native输入处理管道。这些输入处理管道类似独立的非Keras流程代码，被用于与Keras模型对接，并作为Keras模型的一部分被保存和输出

使用预处理层，你可以建立和输出真正的端到端模型：模型将原始图像或者原始的结构化数据作为输入，模型自行完成其数据的特征标准化和特征值索引

## Available preprocessing layers
### Core preprocessing layers
1. TextVectorization layer: turns raw strings into an encoded representation that can be read by an Embedding layer or Dense layer.
2. Normalization layer: performs feature-wise normalize of input features.

## 可用的预处理层
### 核心预处理层
1. TextVectorization layer: 将原始字符串转换为可以被潜入层或稠密层读取的编码
2. Normalization layer: 执行输入特征的标准化/归一化


***
### Structured data preprocessing layers
These layers are for structured data encoding and feature engineering.

1. CategoryEncoding layer: turns integer categorical features into one-hot, multi-hot, or TF-IDF dense representations.
2. Hashing layer: performs categorical feature hashing, also known as the "hashing trick".
3. Discretization layer: turns continuous numerical features into integer categorical features.
4. StringLookup layer: turns string categorical values into integers indices.
5. IntegerLookup layer: turns integer categorical values into integers indices.
6. CategoryCrossing layer: combines categorical features into co-occurrence features. E.g. if you have feature values "a" and "b", it can provide with the combination feature "a and b are present at the same time".


### 结构化数据的预处理层
有些层可以用于对结构化数据进行编码和特征工程：

1. CategoryEncoding layer:将整数类别变量转化为独热，multi-hot(多热?)，或TF-IDF编码表示
2. Hashing layer: 执行类别变量特征哈希
3. Discretization layer: 将连续变量转化为整数类别变量
4. StringLookup layer: 将字符串变量转化为整数indices.
5. IntegerLookup layer: 将整数类别变量转化为整数indices.
6. CategoryCrossing layer: 将类别特征组合形成新的共同类别特征.比如，你有特征"a"和"b"，这可以合成一个共同特征“a和b同时表达”

***
### Image preprocessing layers
These layers are for standardizing the inputs of an image model.

1. Resizing layer: resizes a batch of images to a target size.
2. Rescaling layer: rescales and offsets the values of a batch of image (e.g. go from inputs in the [0, 255] range to inputs in the [0, 1] range.
3. CenterCrop layer: returns a center crop of a batch of images.

### 图像预处理层
有些层可以用于标准化图像模型的输入：

1. Resizing layer: 将批量图像调整到指定大小.
2. Rescaling layer: 将批量图像缩放并偏移（比如从取值范围为\[0-255\]到取值范围\[0-1\]）.
3. CenterCrop layer:返回批量图像的中心裁剪部分.


***
### Image data augmentation layers
These layers apply random augmentation transforms to a batch of images. They are only active during training.

1. RandomCrop layer
2. RandomFlip layer
3. RandomTranslation layer
4. RandomRotation layer
5. RandomZoom layer
6. RandomHeight layer
7. RandomWidth layer

###
图像数据增强层
有些层用于随机增强转化批量图像（通过对同一图片剪切，缩放，旋转等处理来获得新的同类型的图片），这些层只会在训练中激活
1. RandomCrop layer
2. RandomFlip layer
3. RandomTranslation layer
4. RandomRotation layer
5. RandomZoom layer
6. RandomHeight layer
7. RandomWidth layer
***

## The adapt() method
## adapt()方法
Some preprocessing layers have an internal state that must be computed based on a sample of the training data. The list of stateful preprocessing layers is:

1. TextVectorization: holds a mapping between string tokens and integer indices
2. Normalization: holds the mean and standard deviation of the features
3. StringLookup and IntegerLookup: hold a mapping between input values and output indices.
4. CategoryEncoding: holds an index of input values.
5. Discretization: holds information about value bucket boundaries.

Crucially, these layers are non-trainable. Their state is not set during training; it must be set before training, a step called "adaptation".

You set the state of a preprocessing layer by exposing it to training data, via the adapt() method:

一些预处理层会有一个“内部状态”，这一状态是根据训练数据样本计算得出的。这类“状态”预处理层包括：
1. TextVectorization: 保持从字符串tokens到整数indices的映射关系
2. Normalization: 保持特征的均值和标准差
3. StringLookup and IntegerLookup: 保持从输入到输出的映射关系
4. CategoryEncoding: 保持输入值的索引
5. Discretization: 保持value bucket的边界信息

重要的是，这些层是不可训练的。这些状态并不是在训练过程中被设定的。它们必须在一个称之为“调整”的过程（早于训练）中被设定

你可以通过adapt()方法，以将设定状态暴露给训练数据的方式来设定它们

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

data = np.array([[0.1, 0.2, 0.3], [0.8, 0.9, 1.0], [1.5, 1.6, 1.7],])
layer = preprocessing.Normalization()
layer.adapt(data)
normalized_data = layer(data)

print("Features mean: %.2f" % (normalized_data.numpy().mean()))
print("Features std: %.2f" % (normalized_data.numpy().std()))

Features mean: 0.00
Features std: 1.00


***
The adapt() method takes either a Numpy array or a tf.data.Dataset object. In the case of StringLookup and TextVectorization, you can also pass a list of strings:

adapt()方法可以接受一个Numpy数组或tf.data.Dataset对象。在StringLookup和TextVectorization中，你也可以传递一个字符串

In [3]:
data = [
    "ξεῖν᾽, ἦ τοι μὲν ὄνειροι ἀμήχανοι ἀκριτόμυθοι",
    "γίγνοντ᾽, οὐδέ τι πάντα τελείεται ἀνθρώποισι.",
    "δοιαὶ γάρ τε πύλαι ἀμενηνῶν εἰσὶν ὀνείρων:",
    "αἱ μὲν γὰρ κεράεσσι τετεύχαται, αἱ δ᾽ ἐλέφαντι:",
    "τῶν οἳ μέν κ᾽ ἔλθωσι διὰ πριστοῦ ἐλέφαντος,",
    "οἵ ῥ᾽ ἐλεφαίρονται, ἔπε᾽ ἀκράαντα φέροντες:",
    "οἱ δὲ διὰ ξεστῶν κεράων ἔλθωσι θύραζε,",
    "οἵ ῥ᾽ ἔτυμα κραίνουσι, βροτῶν ὅτε κέν τις ἴδηται.",
]
layer = preprocessing.TextVectorization()
layer.adapt(data)
vectorized_text = layer(data)
print(vectorized_text)

tf.Tensor(
[[37 12 25  5  9 20 21  0  0]
 [51 34 27 33 29 18  0  0  0]
 [49 52 30 31 19 46 10  0  0]
 [ 7  5 50 43 28  7 47 17  0]
 [24 35 39 40  3  6 32 16  0]
 [ 4  2 15 14 22 23  0  0  0]
 [36 48  6 38 42  3 45  0  0]
 [ 4  2 13 41 53  8 44 26 11]], shape=(8, 9), dtype=int64)


***
In addition, adaptable layers always expose an option to directly set state via constructor arguments or weight assignment. If the intended state values are known at layer construction time, or are calculated outside of the adapt() call, they can be set without relying on the layer's internal computation. For instance, if external vocabulary files for the TextVectorization, StringLookup, or IntegerLookup layers already exist, those can be loaded directly into the lookup tables by passing a path to the vocabulary file in the layer's constructor arguments.

Here's an example where we instantiate a StringLookup layer with precomputed vocabulary:

另外，调整层总是可以通过结构参数或者权重分配来直接设置状态。如果预期的状态值在构建层时就已经知道，或者可以在adapt之外计算得出，那么这些状态值可以在没有内部计算的前提下被直接设定。比如说，如果存在一个用于TextVectorization, StringLookup, 或者IntegerLookup层的词汇表文件，那么可以在层的构建参数中传递词汇表文件的路径，以此来加载词汇表并直接添加在搜索表中

In [4]:
vocab = ["a", "b", "c", "d"]
data = tf.constant([["a", "c", "d"], ["d", "z", "b"]])
layer = preprocessing.StringLookup(vocabulary=vocab)
vectorized_data = layer(data)
print(vectorized_data)

tf.Tensor(
[[2 4 5]
 [5 1 3]], shape=(2, 3), dtype=int64)


***
## Preprocessing data before the model or inside the model
## 在建模之前或在模型内部预处理数据

There are two ways you could be using preprocessing layers:

**Option 1:** Make them part of the model, like this:

有两种方法可以使用与处理层

**选项1:** 让（预处理层）成为模型的一部分，如 

In [ ]:
inputs = keras.Input(shape=input_shape)
x = preprocessing_layer(inputs)
outputs = rest_of_the_model(x)
model = keras.Model(inputs, outputs)

With this option, preprocessing will happen on device, synchronously with the rest of the model execution, meaning that it will benefit from GPU acceleration. If you're training on GPU, this is the best option for the Normalization layer, and for all image preprocessing and data augmentation layers.

选择这个选项，预处理过程会发生在设备上，与模型执行的其他部分同步，这意味着预处理过程可以享受GPU加速。如果你用GPU进行训练，对归一化层，以及所有的图像预处理层和数据增强层而言都，这是最好的选择
***

**Option 2:** apply it to your tf.data.Dataset, so as to obtain a dataset that yields batches of preprocessed data, like this:
**Option 2:** 应用于你的tf.data.Dataset，这样就可以获得生成批量预处理数据的集，例如：

In [ ]:
dataset = dataset.map(
  lambda x, y: (preprocessing_layer(x), y))

With this option, your preprocessing will happen on CPU, asynchronously, and will be buffered before going into the model.

This is the best option for TextVectorization, and all structured data preprocessing layers. It can also be a good option if you're training on CPU and you use image preprocessing layers.

选择这个选项，你的预处理过程会发生在CPU上并在加入模型前获得缓冲机会

对于TextVectorization和所有的结构化数据而言，这是最好的方式。如果你在CPU上进行训练并且你使用图像预处理层，这些事一个好选择

***

## Benefits of doing preprocessing inside the model at inference time
Even if you go with option 2, you may later want to export an inference-only end-to-end model that will include the preprocessing layers. The key benefit to doing this is that it makes your model portable and it helps reduce the training/serving skew.

## 在预测时，在模型内部进行预处理的好处
即使你选择了option2，你可能稍后需要导出一个只用于预测或者端到包含预处理层的模型。 这样做的主要好处是让模型更佳便携，并且有助于减少训练//serving skew

When all data preprocessing is part of the model, other people can load and use your model without having to be aware of how each feature is expected to be encoded & normalized. Your inference model will be able to process raw images or raw structured data, and will not require users of the model to be aware of the details of e.g. the tokenization scheme used for text, the indexing scheme used for categorical features, whether image pixel values are normalized to \[-1, +1\] or to \[0, 1\], etc. This is especially powerful if you're exporting your model to another runtime, such as TensorFlow.js: you won't have to reimplement your preprocessing pipeline in JavaScript.

当所有数据的预处理是模型的一部分时，其他人可以加载并使用你的模型而不需要关注每一个特征是如何编码和归一化的。你的预测模型可以数据原始图像或者原始结构化数据而不需要模型使用者了解具体细节，比如用于处理文本的分词格式，用于处理分类特征的索引格式，图像像素值归一化范围是\[-1, +1\] 还是 \[0, 1\]等等。如果你需要导出模型到另一个运行环境，如TensorFlow.js时。这将非常有用，因为你不需要在JavaScript中重新实现你的预处理管道

If you initially put your preprocessing layers in your tf.data pipeline, you can export an inference model that packages the preprocessing. Simply instantiate a new model that chains your preprocessing layers and your training model:

如果你一开始就吧预处理层加入你的tf.data管道，你可以导出一个与预处理过程打包的预测模型，轻松地实例化一个链接你的预处理层和训练模型的新模型

In [ ]:
inputs = keras.Input(shape=input_shape)
x = preprocessing_layer(inputs)
outputs = training_model(x)
inference_model = keras.Model(inputs, outputs)

***
## Quick recipes
## 快速实现示例
### Image data augmentation (on-device)
### 图像数据增强（在设备，即将预处理层整合进模型之中）

Note that image data augmentation layers are only active during training (similarly to the Dropout layer).
注意图像数据增强层只在训练过程中激活（dropout层也是如此）

In [5]:
from tensorflow import keras
from tensorflow.keras import layers

# Create a data augmentation stage with horizontal flipping, rotations, zooms
# 创建一个数据增强平移，旋转，缩放的数据增强部分
data_augmentation = keras.Sequential(
    [
        preprocessing.RandomFlip("horizontal"),
        preprocessing.RandomRotation(0.1),
        preprocessing.RandomZoom(0.1),
    ]
)

# Create a model that includes the augmentation stage
# 创建一个模型包含这个增强部分

input_shape = (32, 32, 3)
classes = 10
inputs = keras.Input(shape=input_shape)
# Augment images
#增强图像
x = data_augmentation(inputs)
# Rescale image values to [0, 1]
# 重新缩放图像值到[0, 1]
x = preprocessing.Rescaling(1.0 / 255)(x)
# Add the rest of the model
# 加入模型的其他部分
outputs = keras.applications.ResNet50(
    weights=None, input_shape=input_shape, classes=classes
)(x)
model = keras.Model(inputs, outputs)

You can see a similar setup in action in the example [image classification from scratch.](https://keras.io/examples/vision/image_classification_from_scratch/)

你可以在这个[案例](https://keras.io/examples/vision/image_classification_from_scratch/)的环境设定部分看到类似的做法
***

### Normalizing numerical features
### 数值型特征归一化

In [6]:
# Load some data
# 加载数据
(x_train, y_train), _ = keras.datasets.cifar10.load_data()
x_train = x_train.reshape((len(x_train), -1))
input_shape = x_train.shape[1:]
classes = 10

# Create a Normalization layer and set its internal state using the training data
# 创建归一化层并用训练数据设置他的内部状态
normalizer = preprocessing.Normalization()
normalizer.adapt(x_train)

# Create a model that include the normalization layer
# 创建一个模型，包含归一化层
inputs = keras.Input(shape=input_shape)
x = normalizer(inputs)
outputs = layers.Dense(classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

# Train the model
# 训练模型
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
model.fit(x_train, y_train)

1563/1563 [==============================] - 3s 2ms/step - loss: 2.1763


### Encoding string categorical features via one-hot encoding
### 通过独热编码对字符串类别特征进行编码

In [12]:
# Define some toy data
# 定义一些玩具数据
data = tf.constant(["a", "b", "c", "b", "c", "a"])

# Use StringLookup to build an index of the feature values
# 使用StringLookup来建立一个特征值索引
indexer = preprocessing.StringLookup()
indexer.adapt(data)

# Use CategoryEncoding to encode the integer indices to a one-hot vector
# 使用CategoryEncoding对整数索引进行编码，生成独热向量
encoder = preprocessing.CategoryEncoding(output_mode="binary")
encoder.adapt(indexer(data))

# Convert new test data (which includes unknown feature values)
# 转换新的测试数据（包含未知的特征值）
test_data = tf.constant(["a", "b", "c", "d", "e", ""])
encoded_data = encoder(indexer(test_data))
print(encoded_data)

tf.Tensor(
[[0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]], shape=(6, 5), dtype=float32)


Note that index 0 is reserved for missing values (which you should specify as the empty string ""), and index 1 is reserved for out-of-vocabulary values (values that were not seen during adapt()). You can configure this by using the mask_token and oov_token constructor arguments of StringLookup.

You can see the StringLookup and CategoryEncoding layers in action in the example [structured data classification from scratch.](https://keras.io/examples/structured_data/structured_data_classification_from_scratch/)

注意，0是为缺失值准备的（即空字符串“”），而1是词汇外的输入（无法在adapt()中找到的值）。你可以通过使用StringLookup的构造参数mask_token和oov_token进行设置

你可以在这个[例子](https://keras.io/examples/structured_data/structured_data_classification_from_scratch/)中查看StringLookup层和CategoryEncoding层时如何工作的。
***

### Encoding integer categorical features via one-hot encoding
### 通过独热编码对整数型类别特征进行编码

In [14]:
# Define some toy data
data = tf.constant([10, 20, 20, 10, 30, 0])

# Use IntegerLookup to build an index of the feature values
indexer = preprocessing.IntegerLookup()
indexer.adapt(data)

# Use CategoryEncoding to encode the integer indices to a one-hot vector
encoder = preprocessing.CategoryEncoding(output_mode="binary")
encoder.adapt(indexer(data))

# Convert new test data (which includes unknown feature values)
test_data = tf.constant([10, 10, 20, 50, 60, 0])
encoded_data = encoder(indexer(test_data))
print(encoded_data)

tf.Tensor(
[[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]], shape=(6, 5), dtype=float32)


Note that index 0 is reserved for missing values (which you should specify as the value 0), and index 1 is reserved for out-of-vocabulary values (values that were not seen during adapt()). You can configure this by using the mask_value and oov_value constructor arguments of IntegerLookup.

You can see the IntegerLookup and CategoryEncoding layers in action in the example [structured data classification from scratch.](https://keras.io/examples/structured_data/structured_data_classification_from_scratch/)

注意，0是为缺失值准备的（你可以将其指定为0），而1是词汇外的输入（无法在adapt()中找到的值）。你可以通过使用IntegerLookup的构造参数mask_token和oov_token进行设置

你可以在这个[例子](https://keras.io/examples/structured_data/structured_data_classification_from_scratch/)中查看StringLookup层和CategoryEncoding层时如何工作的。
***

# Note 学习笔记

In the above two examples, if index 0 is reserved for missing values and index 1 is reserved for out-of-vocabulary values, which is the out-of-vocabulary values? 

在上述两个例子中，如果索引0代表缺失值，索引1代表字典外的值，那么谁代表字典内的值？

### Applying the hashing trick to an integer categorical feature
### 对整数型类别特征应用特征哈希

If you have a categorical feature that can take many different values (on the order of 10e3 or higher), where each value only appears a few times in the data, it becomes impractical and ineffective to index and one-hot encode the feature values. Instead, it can be a good idea to apply the "hashing trick": hash the values to a vector of fixed size. This keeps the size of the feature space manageable, and removes the need for explicit indexing.

如果你有一个类别特征含有很多类别（1000以上），并且每一种类别只在数据集中出现了几次，那么对特征值进行索引和独特变量就很没必要且效果不佳。相对的，应用“特征哈希”就成为了一个不错的主意：将类别值转换为固定长度的向量。这一方面保证了特征空间大小的可控性，另一方面又消除了对显性索引的需求

In [7]:
# Sample data: 10,000 random integers with values between 0 and 100,000
# 样本数据：10000个取值在0刀100000之间的随机整数
data = np.random.randint(0, 100000, size=(10000, 1))

# Use the Hashing layer to hash the values to the range [0, 64]
# 使用哈希层来转换样本值到[0, 64]
hasher = preprocessing.Hashing(num_bins=64, salt=1337)

# Use the CategoryEncoding layer to one-hot encode the hashed values
# 利用CategoryEncoding层来进行独热编码哈希值
encoder = preprocessing.CategoryEncoding(max_tokens=64, output_mode="binary")
encoded_data = encoder(hasher(data))
print(encoded_data.shape)

(10000, 64)


***
### Encoding text as a sequence of token indices
### 将文本编码为分词序列
This is how you should preprocess text to be passed to an Embedding layer.
这一部分展示了如何预处理文本并传递给嵌入层

In [8]:
# Define some text data to adapt the layer
# 定义文本数据，通过adapt（）设置层状态
data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)
# Instantiate TextVectorization with "int" output_mode
# 实例化TextVectorization with（输出模式为整数）
text_vectorizer = preprocessing.TextVectorization(output_mode="int")
# Index the vocabulary via `adapt()`
#通过adapt（）索引词汇表
text_vectorizer.adapt(data)

# You can retrieve the vocabulary we indexed via get_vocabulary()
# 通过get_vocabulary()来检索词汇表
vocab = text_vectorizer.get_vocabulary()
print("Vocabulary:", vocab)

# Create an Embedding + LSTM model
# 创建一个嵌入层+长短期记忆网络模型
inputs = keras.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = layers.Embedding(input_dim=len(vocab), output_dim=64)(x)
outputs = layers.LSTM(1)(x)
model = keras.Model(inputs, outputs)

# Call the model on test data (which includes unknown tokens)
# 通过测试数据（含有未知的分词）调用模型
test_data = tf.constant(["The Brain is deeper than the sea"])
test_output = model(test_data)

Vocabulary: ['', '[UNK]', 'the', 'side', 'you', 'with', 'will', 'wider', 'them', 'than', 'sky', 'put', 'other', 'one', 'is', 'for', 'ease', 'contain', 'by', 'brain', 'beside', 'and']


# Note 学习笔记

I check the test_output of the above example. It returns a value. But I don't know the meaning of the value. what is the goal of the model?

我查看了上面示例的输出，这个模型返回了一个值。但是我不明白这个值意味着什么？这个模型的目的是什么？

You can see the TextVectorization layer in action, combined with an Embedding mode, in the example [text classification from scratch.](https://keras.io/examples/nlp/text_classification_from_scratch/)

Note that when training such a model, for best performance, you should use the TextVectorization layer as part of the input pipeline (which is what we do in the text classification example above).

你可以在这个[示例](https://keras.io/examples/nlp/text_classification_from_scratch/)中查看层在嵌入模式下的工作情况

注意，当训练这样一个模型时，为了最好的表现，你应该将TextVectorization层作为输入管道的一部分（即我们在文本分类示例中所做的）
***

### Encoding text as a dense matrix of ngrams with multi-hot encoding
### 用multi-hot编码对文本编码为ngrams稠密矩阵
This is how you should preprocess text to be passed to a Dense layer.
这一部分展示了如何预处理文本并传递给稠密层

In [18]:
# Define some text data to adapt the layer
data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)
# Instantiate TextVectorization with "binary" output_mode (multi-hot)
# and ngrams=2 (index all bigrams)
text_vectorizer = preprocessing.TextVectorization(output_mode="binary", ngrams=2)
# Index the bigrams via `adapt()`
text_vectorizer.adapt(data)

print(
    "Encoded text:\n",
    text_vectorizer(["The Brain is deeper than the sea"]).numpy(),
    "\n",
)

# Create a Dense model
inputs = keras.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

# Call the model on test data (which includes unknown tokens)
test_data = tf.constant(["The Brain is deeper than the sea"])
test_output = model(test_data)

print("Model output:", test_output)

Encoded text:
 [[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]] 

Model output: tf.Tensor([[-0.6287689]], shape=(1, 1), dtype=float32)


***
### Encoding text as a dense matrix of ngrams with TF-IDF weighting
### 用TF-IDF权重编码对文本编码为ngrams稠密矩阵
This is an alternative way of preprocessing text before passing it to a Dense layer.
这一部分展示了一个预处理文本并传递给稠密层的替代方法

In [19]:
# Define some text data to adapt the layer
data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)
# Instantiate TextVectorization with "tf-idf" output_mode
# (multi-hot with TF-IDF weighting) and ngrams=2 (index all bigrams)
text_vectorizer = preprocessing.TextVectorization(output_mode="tf-idf", ngrams=2)
# Index the bigrams and learn the TF-IDF weights via `adapt()`
text_vectorizer.adapt(data)

print(
    "Encoded text:\n",
    text_vectorizer(["The Brain is deeper than the sea"]).numpy(),
    "\n",
)

# Create a Dense model
inputs = keras.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

# Call the model on test data (which includes unknown tokens)
test_data = tf.constant(["The Brain is deeper than the sea"])
test_output = model(test_data)
print("Model output:", test_output)

Encoded text:
 [[8.04719   1.6945957 0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        1.0986123 1.0986123 1.0986123 0.        0.
  0.        0.        0.        0.        0.        0.        0.
  1.0986123 0.        0.        0.        0.        0.        0.
  0.        1.0986123 1.0986123 0.        0.        0.       ]] 

Model output: tf.Tensor([[-0.27005047]], shape=(1, 1), dtype=float32)
